<a href="https://colab.research.google.com/github/enzoampil/tito-joker/blob/train_model2/experiments/Jokes_GPT2_Finetuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tito Joker v2

## Updates:
1. Reduce sequence length to 50 (closer to typical joke length)
2. Double batch size from 2 to 4 (can afford it now due to 1)
3. Special tokens were formally added to the vocabulary
4. Still fine-tuned on OpenAI-GPT2 w/ the same dataset (but w/ `<eos>` removed in favor of default end of sequence special token of GPT2 - `<|endoftext|>`)

## Note that the experiment notebooks have ending numbers corresponding to the model's number (this is model2)

In [14]:
!pip install transformers

     |████████████████████████████████| 552kB 8.5MB/s 
     |████████████████████████████████| 1.0MB 56.3MB/s 
     |████████████████████████████████| 3.7MB 49.7MB/s 
     |████████████████████████████████| 870kB 59.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=285ccec2c9633d1f0431a7cf062431cf23173c4fbecf7235e491d834f6d830cb
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 23566 (delta 27), reused 32 (delta 13), pack-reused 23498
Receiving objects: 100% (23566/23566), 14.04 MiB | 6.69 MiB/s, done.
Resolving deltas: 100% (16690/16690), done.


In [3]:
!git clone https://github.com/enzoampil/tito-joker.git

fatal: destination path 'tito-joker' already exists and is not an empty directory.


## Download raw jokes dataset

In [8]:
!wget https://storage.googleapis.com/joke-generator-model1/short-jokes.zip && unzip short-jokes.zip

--2020-04-05 15:31:24--  https://storage.googleapis.com/joke-generator-model1/short-jokes.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10679873 (10M) [application/zip]
Saving to: ‘short-jokes.zip’

short-jokes.zip     100%[===================>]  10.18M  --.-KB/s    in 0.03s   

2020-04-05 15:31:24 (300 MB/s) - ‘short-jokes.zip’ saved [10679873/10679873]

unzip:  cannot find or open tito-joker/data/short-jokes.zip, tito-joker/data/short-jokes.zip.zip or tito-joker/data/short-jokes.zip.ZIP.


## Process the raw jokes

In [5]:
%cd tito-joker

/content/tito-joker


## Prepare dataset

In [0]:
!python ./src/utils/process_jokes.py ../shortjokes.csv ./data/riddle_jokes.txt

In [10]:
ls data

riddle_jokes.txt  shortjokes_sample100.csv


In [0]:
#from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
#from pathlib import Path

In [0]:
from transformers import GPT2Model, GPT2Config

## Save tokenizer w/ new tokens

In [0]:
!mkdir models
!mkdir models/pretrained
!mkdir models/finetuned

In [18]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
new_tokens = ['<soq>', '<eoq>']
num_added_toks = tokenizer.add_tokens(new_tokens)

In [19]:
tokenizer.save_pretrained('./models/pretrained')

('./models/pretrained/vocab.json',
 './models/pretrained/merges.txt',
 './models/pretrained/special_tokens_map.json',
 './models/pretrained/added_tokens.json')

In [0]:
import json
with open('./models/pretrained/config.json', 'w') as f:
    json.dump(GPT2Config().to_dict(), f)

## Finetune the GPT2 model

In [25]:
!python ../transformers/examples/run_language_modeling.py\
    --overwrite_output_dir\ # Overwrite existing output dir
    --output_dir=./models/finetuned \
    --model_type=gpt2\
    --model_name_or_path=gpt2\
    --tokenizer_name ./models/pretrained \
    --do_train\
    --train_data_file=./data/riddle_jokes.txt \
    --per_gpu_train_batch_size 4 \
    --block_size 50 # Set this arbitrarily for now. But most jokes will be within this range. We don't want to weight super long jokes anyway.\

04/05/2020 15:39:00 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/05/2020 15:39:01 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
04/05/2020 15:39:01 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 2,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encod

## Test results

In [36]:
%cd /content

/content


In [41]:
!python transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=tito-joker/models/finetuned/ \
    --length=50 \
    --stop_token='<|endoftext|>' \
    --num_return_sequences 100

04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   Model name 'tito-joker/models/finetuned/' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'tito-joker/models/finetuned/' is a path, a model identifier, or url to a directory containing tokenizer files.
04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/vocab.json
04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/merges.txt
04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/added_tokens.json
04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/special_tokens_map.json
04/05/2020 16:10:56 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/tokenizer_config.json
04/05/2020 16:10:57 - INFO - transformers.configuration_utils - 

## Save results to gcs

In [29]:
%cd models/finetuned 

/content/tito-joker/models/finetuned


In [30]:
ls

added_tokens.json  checkpoint-3500/  checkpoint-6000/   special_tokens_map.json
checkpoint-1000/   checkpoint-4000/  checkpoint-6500/   tokenizer_config.json
checkpoint-1500/   checkpoint-4500/  checkpoint-7000/   training_args.bin
checkpoint-2000/   checkpoint-500/   config.json        vocab.json
checkpoint-2500/   checkpoint-5000/  merges.txt
checkpoint-3000/   checkpoint-5500/  pytorch_model.bin


In [31]:
!zip model2.zip added_tokens.json config.json merges.txt pytorch_model.bin special_tokens_map.json tokenizer_config.json training_args.bin vocab.json

ERROR! Session/line number was not unique in database. History logging moved to new session 61
  adding: added_tokens.json (deflated 22%)
  adding: config.json (deflated 57%)
  adding: merges.txt (deflated 53%)
  adding: pytorch_model.bin (deflated 16%)
  adding: special_tokens_map.json (deflated 52%)
  adding: tokenizer_config.json (deflated 48%)
  adding: training_args.bin (deflated 37%)
  adding: vocab.json (deflated 63%)


In [33]:
ls -l

total 988956
-rw-r--r-- 1 root root        32 Apr  5 15:50 added_tokens.json
drwxr-xr-x 2 root root      4096 Apr  5 15:40 checkpoint-1000/
drwxr-xr-x 2 root root      4096 Apr  5 15:41 checkpoint-1500/
drwxr-xr-x 2 root root      4096 Apr  5 15:42 checkpoint-2000/
drwxr-xr-x 2 root root      4096 Apr  5 15:43 checkpoint-2500/
drwxr-xr-x 2 root root      4096 Apr  5 15:43 checkpoint-3000/
drwxr-xr-x 2 root root      4096 Apr  5 15:44 checkpoint-3500/
drwxr-xr-x 2 root root      4096 Apr  5 15:45 checkpoint-4000/
drwxr-xr-x 2 root root      4096 Apr  5 15:46 checkpoint-4500/
drwxr-xr-x 2 root root      4096 Apr  5 15:39 checkpoint-500/
drwxr-xr-x 2 root root      4096 Apr  5 15:46 checkpoint-5000/
drwxr-xr-x 2 root root      4096 Apr  5 15:47 checkpoint-5500/
drwxr-xr-x 2 root root      4096 Apr  5 15:48 checkpoint-6000/
drwxr-xr-x 2 root root      4096 Apr  5 15:49 checkpoint-6500/
drwxr-xr-x 2 root root      4096 Apr  5 15:49 checkpoint-7000/
-rw-r--r-- 1 root root      1307 Apr  5 15

In [0]:
from google.colab import auth
auth.authenticate_user()

In [35]:
!gsutil cp model2.zip gs://joke-generator-model1/

Copying file://model2.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/441.7 MiB.                                    
